# 패키지 임포트

In [1]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import sys
sys.path.append('C:/Users/User/Desktop/WELFake')
from data_preprocessing import *
from transformer_build import  *

# 데이터 로드

In [2]:
df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

# 전처리

In [3]:
df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

X=X['title']+' '+X['text']

X = X.apply(clean_text)
X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)
        
X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
to_txt=x_train+x_test

In [4]:
y=y_train+y_test

In [5]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [6]:
lst=[]
for i in range(16000):
    lst.append(len(x_train[i].split()))

In [7]:
max(lst)

1995

In [8]:
sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 1000

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

In [9]:
tmp = [[x,y] for x, y in zip(x_train, y_train)]
random.shuffle(tmp)
x_train = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]
x_train=np.array(x_train)
y_train=np.array(y_train)

In [10]:
x_val=x_train[:1000]
x_train=x_train[1000:]
y_val=y_train[:1000]
y_train=y_train[1000:]
y_train = np.array(y_train)
y_test = np.array(y_test)

# 모델 빌드

In [11]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [12]:
embedding_dim = 128  # 각 단어의 임베딩 벡터의 차원   #128 #256 #512 #1024
num_heads = 1  # 어텐션 헤드의 수
dff = 128 # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기 #32 #64 #128 #256
num_transformer_blocks = 1

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    x = transformer_block(x)

x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
# x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000)]            0         
                                                                 
 token_and_position_embeddin  (None, 1000, 128)        22856192  
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 1000, 128)        99584     
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 128)               0     

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=128,callbacks=callbacks, epochs=10, validation_data=(x_val, y_val))

print("테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

Epoch 1/10
 83/118 [====================>.........] - ETA: 1s - loss: 0.5160 - accuracy: 0.7108

In [ ]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model_5.h5', custom_objects=custom_objects)
model.evaluate(x_test, y_test)